In [82]:
import numpy as np
import csv
import h5py
import pickle
import pandas as pd
import networkx as nx
from sklearn.cluster import k_means
from sklearn.cluster import KMeans
import random
from sklearn.cluster import SpectralClustering
import warnings
from networkx.algorithms.community.centrality import girvan_newman
warnings.filterwarnings('ignore')
import dask.dataframe as dd
import dask
import dask_ml.cluster as dmc
import dask.array as da
import distributed
import msgpack
from dask.distributed import progress

In [12]:
from dask.distributed import Client

client = Client(n_workers=4, threads_per_worker=2, memory_limit="3GB")
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 11.18 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:50501,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 11.18 GiB
Comm: tcp://127.0.0.1:50522,Total threads: 2
Dashboard: http://127.0.0.1:50528/status,Memory: 2.79 GiB
Nanny: tcp://127.0.0.1:50504,


In [13]:
df1=pd.read_csv(r"large_twitch_edges.csv")
df2=pd.read_csv(r"large_twitch_features.csv")

edges_array=df1.to_numpy()
features_array=df2.to_numpy()

In [14]:
%%time
Twitch_Graph=nx.Graph()
for i in edges_array:
    Twitch_Graph.add_edge(i[0],i[1])

CPU times: total: 31.5 s
Wall time: 33.2 s


In [107]:
def clustering(Graph, number_of_clusters, type_of_graph, labels):
    print(Graph)
    if type_of_graph == 0:
        kmeans_model = KMeans(n_clusters=number_of_clusters)
        labels = kmeans_model.fit_predict(nx.to_numpy_array(Graph))
    else:
        labels=labels
    for node, label in zip(Graph.nodes, labels):                      
        Graph.nodes[node]['kmeans_cluster'] = label
    dict_=[]                                                
    for i in set(labels):                                       
        dict_.append(set())                                              
    for node in Graph.nodes:                            
        dict_[Graph.nodes[node]['kmeans_cluster']].add(node)
    for i in range(len(dict_)):
        dict_[i]=set(dict_[i])
    modularity=nx.community.modularity(Graph, dict_)
    print('The modularity is:', modularity)
    for i in range(0, number_of_clusters):
        print('The length of cluster  ', i, '   is', len(dict_[i]))
    return dict_

def node_affiliation(Graph, type_of_graph):
    if type_of_graph == 0:
        random.seed(1)
        for i in Graph.nodes:
            r=random.randrange(0,1000)
            if (r<500):
                Graph.nodes[i]['affiliation']=0
            else:
                Graph.nodes[i]['affiliation']=1
    elif type_of_graph == 1:
        for i in Graph.nodes:
            if(features_array[i][8]==1):
                Graph.nodes[i]['affiliation']=1
            else:
                Graph.nodes[i]['affiliation']=0


def graph_fairness(Graph):
    c=0
    for i in Graph.nodes:
         if(Graph.nodes[i]['affiliation']==1):
                c+=1
    aff_percent=c/len(Graph.nodes)
    non_aff_percent=1-aff_percent
    print(100*aff_percent, "% of the users are affiliated")
    return 100*aff_percent

def find_cluster_fairness(Graph, dict_, print_method):
    num_of_aff=[0 for i in range(len(dict_))]

    k=0
    for i in dict_:
        for j in i:
            if(Graph.nodes[j]['affiliation']==1):
                num_of_aff[k]+=1
        k+=1
    temp=0
    for i in range(len(dict_)):
        temp+=num_of_aff[i]/len(dict_[i])
        if print_method == 1:
            print("Cluster ",i," has ",100*num_of_aff[i]/len(dict_[i]),"% affiliated streamers")
    if print_method == 1:
        print("Clusters have an average of ", 100*temp/len(dict_),"% affiliated streamers")
    return num_of_aff

def improve_fairness(Graph, dict_, num_of_aff, graph_fairness, divergence, type_of_graph):
    k=0
    for i in range(len(dict_)):
        for j in range(len(dict_)):
            while (100*num_of_aff[i]/len(dict_[i])<graph_fairness-divergence and 100*num_of_aff[j]/len(dict_[j])>graph_fairness+divergence):
                print(k,'           ', i, j)
                k+=1
                flag_1=False
                for n1 in dict_[i]: 
                    if type_of_graph == 0:
                        t=dict_[j]
                        t.add(n1)
                        sub=Graph.subgraph(t)
                        if (Graph.nodes[n1]['affiliation']==0 and nx.is_connected(sub)):
                            dict_[i].remove(n1)
                            dict_[j].add(n1)
                            print('ok')
                            flag_1=True
                    elif type_of_graph == 1:
                        if (Graph.nodes[n1]['affiliation']==0):
                            dict_[i].remove(n1)
                            dict_[j].add(n1)
                            print('ok')
                            flag_1=True
                    if (flag_1):
                        print("1 FLAG 1")
                        break
                num_of_aff=find_cluster_fairness(Graph, dict_, 0)  


In [16]:
%%time
random.seed(1)
p=0.3
q=0.02
sizes=[400, 100, 300, 200]
P2=[[] for i in range(0,4)]
for i in range(0,4):
    for j in range(0,4):
        if (i!=j):
            P2[j].append(random.randrange(0,1000)/10000)
        else:
            P2[j].append(random.randrange(200,1000)/2000)
for i in range(0,4):
    for j in range(i,4):
        P2[j][i]=P2[i][j]

P1=[[] for i in range(0,4)]
for i in range(0,4):
    for j in range(0,4):
        if (i!=j):
            P1[j].append(q)
        else:
            P1[j].append(p)
synthetic_1=nx.stochastic_block_model(sizes,P1,seed=1)
synthetic_2=nx.stochastic_block_model(sizes,P2,seed=1)
print(synthetic_1)
print(synthetic_2)

Graph named 'stochastic_block_model' with 1000 nodes and 51898 edges
Graph named 'stochastic_block_model' with 1000 nodes and 55593 edges
CPU times: total: 391 ms
Wall time: 413 ms


In [76]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(edges_array)
dask_array = da.from_array(scaled_data, chunks=(20000,2))  
daskar = dask_array.persist()
print(daskar)

dask.array<array, shape=(6797557, 2), dtype=float64, chunksize=(20000, 2), chunktype=numpy.ndarray>


In [77]:
%%time
kmeans_model = dmc.KMeans(n_clusters=4)
kmeans_model.fit(daskar)
dask_kmeans_labels = kmeans_model.labels_
kmeans_labels = dask_kmeans_labels.compute()

CPU times: total: 3min 28s
Wall time: 5min 12s


In [78]:
%%time
Spectral_clustering=dmc.SpectralClustering(n_clusters=4, n_components=100, persist_embedding=True)
dask_Spectral_labels = Spectral_clustering.fit_predict(dask_array)
Spectral_labels=dask_Spectral_labels.compute()

CPU times: total: 9min 20s
Wall time: 20min 7s


In [115]:
%%time
label_propagation=nx.community.label_propagation_communities(Twitch_Graph)
label_propagation_modularity=nx.community.modularity(Twitch_Graph, label_propagation)
print(label_propagation_modularity)
print(len(label_propagation))
label_propagation_clusters=list(label_propagation)

0.03567630678997064
79
CPU times: total: 6min 27s
Wall time: 6min 40s


In [105]:
%%time
dict_kmeans= clustering(Twitch_Graph, 4, 1, kmeans_labels)
node_affiliation(Twitch_Graph, 1)
Twitch_Graph_fair=graph_fairness(Twitch_Graph)
kmeans_aff=find_cluster_fairness(Twitch_Graph, dict_kmeans, 1)
improve_fairness(Twitch_Graph, dict_kmeans, kmeans_aff, Twitch_Graph_fair, 0.8, 1)
find_cluster_fairness(Twitch_Graph, dict_kmeans, 1)

Graph with 168114 nodes and 6797557 edges
The modularity is: 0.0158286905068509
The length of cluster   0    is 40513
The length of cluster   1    is 44646
The length of cluster   2    is 41170
The length of cluster   3    is 41785
48.50637067704058 % of the users are affiliated
Cluster  0  has  47.8685853923432 % affiliated streamers
Cluster  1  has  49.453478475115354 % affiliated streamers
Cluster  2  has  47.61233908185572 % affiliated streamers
Cluster  3  has  48.99365801124806 % affiliated streamers
Clusters have an average of  48.48201524014058 % affiliated streamers
0             2 1
ok
1 FLAG 1
1             2 1
ok
1 FLAG 1
2             2 1
ok
1 FLAG 1
3             2 1
ok
1 FLAG 1
4             2 1
ok
1 FLAG 1
5             2 1
ok
1 FLAG 1
6             2 1
ok
1 FLAG 1
7             2 1
ok
1 FLAG 1
8             2 1
ok
1 FLAG 1
9             2 1
ok
1 FLAG 1
10             2 1
ok
1 FLAG 1
11             2 1
ok
1 FLAG 1
12             2 1
ok
1 FLAG 1
13             2 1
ok
1 F

[19393, 22140, 19602, 20472]

In [117]:
%%time
label_propagation_clusters=list(label_propagation)
node_affiliation(Twitch_Graph, 1)
Twitch_Graph_fair=graph_fairness(Twitch_Graph)
lab_prop_aff=find_cluster_fairness(Twitch_Graph, label_propagation_clusters, 1)
improve_fairness(Twitch_Graph, label_propagation_clusters, lab_prop_aff, Twitch_Graph_fair, 0.34, 1)
find_cluster_fairness(Twitch_Graph, label_propagation_clusters, 1)

48.50637067704058 % of the users are affiliated
Cluster  0  has  48.853464592013296 % affiliated streamers
Cluster  1  has  35.69767441860465 % affiliated streamers
Cluster  2  has  45.08431434934776 % affiliated streamers
Cluster  3  has  59.375 % affiliated streamers
Cluster  4  has  0.0 % affiliated streamers
Cluster  5  has  39.02053712480253 % affiliated streamers
Cluster  6  has  41.202672605790646 % affiliated streamers
Cluster  7  has  0.0 % affiliated streamers
Cluster  8  has  0.0 % affiliated streamers
Cluster  9  has  0.0 % affiliated streamers
Cluster  10  has  0.0 % affiliated streamers
Cluster  11  has  0.0 % affiliated streamers
Cluster  12  has  0.0 % affiliated streamers
Cluster  13  has  0.0 % affiliated streamers
Cluster  14  has  0.0 % affiliated streamers
Cluster  15  has  0.0 % affiliated streamers
Cluster  16  has  50.0 % affiliated streamers
Cluster  17  has  0.0 % affiliated streamers
Cluster  18  has  0.0 % affiliated streamers
Cluster  19  has  50.0 % affili

142             1 3
ok
1 FLAG 1
143             1 3
ok
1 FLAG 1
144             1 3
ok
1 FLAG 1
145             1 3
ok
1 FLAG 1
146             1 3
ok
1 FLAG 1
147             1 3
ok
1 FLAG 1
148             1 3
ok
1 FLAG 1
149             1 3
ok
1 FLAG 1
150             1 3
ok
1 FLAG 1
151             1 3
ok
1 FLAG 1
152             1 3
ok
1 FLAG 1
153             1 3
ok
1 FLAG 1
154             1 3
ok
1 FLAG 1
155             1 3
ok
1 FLAG 1
156             1 16
ok
1 FLAG 1
157             1 19
ok
1 FLAG 1
158             1 24
ok
1 FLAG 1
159             1 24
ok
1 FLAG 1
160             1 33
ok
1 FLAG 1
161             1 33
ok
1 FLAG 1
162             1 33
ok
1 FLAG 1
163             1 34
ok
1 FLAG 1
164             1 37
ok
1 FLAG 1
165             1 37
ok
1 FLAG 1
166             1 37
ok
1 FLAG 1
167             1 38
ok
1 FLAG 1
168             1 42
ok
1 FLAG 1
169             1 48
ok
1 FLAG 1
170             1 51
ok
1 FLAG 1
171             1 51
ok
1 FLAG 1
172             1 51
ok


[78210,
 921,
 1417,
 361,
 0,
 247,
 370,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 2,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 2,
 1,
 0,
 0,
 2,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0]

In [108]:
%%time
dict_spectral= clustering(Twitch_Graph, 4, 1, Spectral_labels)
node_affiliation(Twitch_Graph, 1)
Twitch_Graph_fair=graph_fairness(Twitch_Graph)
spectral_aff=find_cluster_fairness(Twitch_Graph, dict_spectral, 1)
improve_fairness(Twitch_Graph, dict_spectral, spectral_aff, Twitch_Graph_fair, 0.9, 1)
find_cluster_fairness(Twitch_Graph, dict_spectral, 1)

Graph with 168114 nodes and 6797557 edges
The modularity is: 0.01577789908121674
The length of cluster   0    is 44283
The length of cluster   1    is 42103
The length of cluster   2    is 39742
The length of cluster   3    is 41986
48.50637067704058 % of the users are affiliated
Cluster  0  has  49.44786938554299 % affiliated streamers
Cluster  1  has  48.99888368999834 % affiliated streamers
Cluster  2  has  47.57435458708671 % affiliated streamers
Cluster  3  has  47.90168151288525 % affiliated streamers
Clusters have an average of  48.480697293878315 % affiliated streamers
0             2 0
ok
1 FLAG 1
1             2 0
ok
1 FLAG 1
2             2 0
ok
1 FLAG 1
3             2 0
ok
1 FLAG 1
4             2 0
ok
1 FLAG 1
5             2 0
ok
1 FLAG 1
6             2 0
ok
1 FLAG 1
7             2 0
ok
1 FLAG 1
8             2 0
ok
1 FLAG 1
9             2 0
ok
1 FLAG 1
10             2 0
ok
1 FLAG 1
11             2 0
ok
1 FLAG 1
12             2 0
ok
1 FLAG 1
13             2 0
ok
1

[21897, 20630, 18907, 20112]

In [109]:
%%time
labels = 0
dict_synth_1= clustering(synthetic_1, 4, 0, labels)
node_affiliation(synthetic_1, 0)
synth_1_fair=graph_fairness(synthetic_1)
synth_1_aff=find_cluster_fairness(synthetic_1, dict_synth_1, 1)
improve_fairness(synthetic_1, dict_synth_1, synth_1_aff, synth_1_fair, 0.5, 0)
find_cluster_fairness(synthetic_1, dict_synth_1, 1)

Graph named 'stochastic_block_model' with 1000 nodes and 51898 edges
The modularity is: 0.49418921900539003
The length of cluster   0    is 300
The length of cluster   1    is 400
The length of cluster   2    is 100
The length of cluster   3    is 200
51.800000000000004 % of the users are affiliated
Cluster  0  has  48.666666666666664 % affiliated streamers
Cluster  1  has  54.0 % affiliated streamers
Cluster  2  has  49.0 % affiliated streamers
Cluster  3  has  53.5 % affiliated streamers
Clusters have an average of  51.29166666666667 % affiliated streamers
0             0 1
ok
1 FLAG 1
1             0 1
ok
1 FLAG 1
2             0 1
ok
1 FLAG 1
3             0 1
ok
1 FLAG 1
4             0 1
ok
1 FLAG 1
5             0 1
ok
1 FLAG 1
6             0 1
ok
1 FLAG 1
7             0 1
ok
1 FLAG 1
8             0 1
ok
1 FLAG 1
9             0 1
ok
1 FLAG 1
10             0 1
ok
1 FLAG 1
11             0 1
ok
1 FLAG 1
12             0 1
ok
1 FLAG 1
13             0 1
ok
1 FLAG 1
14         

[146, 232, 49, 107]

In [110]:
%%time
labels = 0
dict_synth_2= clustering(synthetic_2, 4, 0, labels)
node_affiliation(synthetic_2, 0)
synth_2_fair=graph_fairness(synthetic_2)
synth_2_aff=find_cluster_fairness(synthetic_2, dict_synth_2, 1)
improve_fairness(synthetic_2, dict_synth_2, synth_2_aff, synth_2_fair, 0.5, 0)
find_cluster_fairness(synthetic_2, dict_synth_2, 1)

Graph named 'stochastic_block_model' with 1000 nodes and 55593 edges
The modularity is: 0.27032389639999443
The length of cluster   0    is 103
The length of cluster   1    is 397
The length of cluster   2    is 300
The length of cluster   3    is 200
51.800000000000004 % of the users are affiliated
Cluster  0  has  50.48543689320388 % affiliated streamers
Cluster  1  has  53.65239294710327 % affiliated streamers
Cluster  2  has  48.666666666666664 % affiliated streamers
Cluster  3  has  53.5 % affiliated streamers
Clusters have an average of  51.57612412674346 % affiliated streamers
0             0 1
ok
1 FLAG 1
1             0 1
ok
1 FLAG 1
2             2 1
ok
1 FLAG 1
3             2 1
ok
1 FLAG 1
4             2 1
ok
1 FLAG 1
5             2 1
ok
1 FLAG 1
6             2 1
ok
1 FLAG 1
7             2 1
ok
1 FLAG 1
8             2 1
ok
1 FLAG 1
9             2 1
ok
1 FLAG 1
10             2 1
ok
1 FLAG 1
11             2 1
ok
1 FLAG 1
12             2 1
ok
1 FLAG 1
13             2

[52, 230, 146, 107]